# Math Square Puzzles

## Tools

### Processing / Transcription

In [ ]:
from PIL import Image, ImageDraw, ImageFont

In [ ]:
import numpy as np

In [ ]:
import pytesseract

In [ ]:
def axis_gridding(ax, minwidth=0):
    streak_indices = np.argwhere(np.diff(ax)).ravel() # excluding start and end
    streak_lengths = np.diff(streak_indices)
    streak_centers = streak_indices[:-1] + streak_lengths/2
    streak_centers = streak_centers[streak_lengths > ax.size * minwidth]
    char_centers = streak_centers[0::2]
    n = len(char_centers)
    grid_spacing, grid_offset = np.polyfit(np.arange(n), char_centers, 1)
    return n, grid_spacing, grid_offset

def transcribe_grid(img, threashold=1/6, minwidth=0, verbose=False):
    img = img.convert('L')
    arr = np.array(img)
    bin_arr = arr < 128
    
    nx, dx, ox = axis_gridding(np.sum(bin_arr, axis=0) > threashold * bin_arr.shape[1], minwidth=minwidth)
    ny, dy, oy = axis_gridding(np.sum(bin_arr, axis=1) > threashold * bin_arr.shape[0], minwidth=minwidth)
    nx, dx = 2*nx, dx/2
    ny, dy = 2*ny, dy/2
    if verbose:
        print(f"{nx} columns, spaced {dx}, starting at {ox}")
        print(f"{ny} rows, spaced {dy}, starting at {oy}")
    #return None, (nx, dx, ox), (ny, dy, oy)

    grid = []
    for i in range(ny):
        grid.append([])
        y0 = max(0, oy + i * dy - dy//2 + 3)
        y1 = min(img.size[1], oy + i * dy + dy//2 - 3)
        for j in range(nx):
            x0 = max(0, ox + j * dx - dx//2 + 3)
            x1 = min(img.size[0], ox + j * dx + dx//2 - 3)
            cimg = img.crop((x0, y0, x1, y1))
            if np.sum(np.array(cimg) < 250) == 0 or np.sum(np.array(cimg) > 10) == 0:
                c = ''
            else:
                c = pytesseract.image_to_string(cimg, config='-c tessedit_char_whitelist=0123456789x+-÷ --psm 10').strip()
                #c = cimg
            grid[-1].append(c)
            if verbose:
                print(f"Processed row {i} / {ny}, column {j} / {nx}: {c}")
    return grid, (nx, dx, ox), (ny, dy, oy)

In [ ]:
def check_grid(grid_img, nx, dx, ox, ny, dy, oy):
    check_grid_img = grid_img.copy()
    draw_check_grid_img = ImageDraw.Draw(check_grid_img)
    for i in range(nx):
        draw_check_grid_img.line((ox+dx*i,0,ox+dx*i,check_grid_img.size[1]), fill=(255,0,0), width=1)
    for i in range(nx):
        draw_check_grid_img.line((0,oy+dy*i,check_grid_img.size[0],oy+dy*i), fill=(255,0,0), width=1)
    return check_grid_img

### Equation Extractions / Solving

In [ ]:
import z3

In [ ]:
import operator

In [ ]:
def add_grid_vars(grid):
    return [
        [
            f"x_{i}_{j}" if i%2==0 and j%2==0 and not c else c.replace('x','*') 
            for j,c in enumerate(row)
        ] 
        for i,row in enumerate(grid)
    ]

In [ ]:
def extract_exprs(grid):
    lrows, lcols = len(grid), len(grid[0])
    exprs = []
    for i in range(lrows//2):
        exprs.append([grid[2*i][j] for j in range(lcols)])
    for i in range(lcols//2):
        exprs.append([grid[j][2*i] for j in range(lrows)])
    return exprs

In [ ]:
def parse_z3_exprs(exprs, no_remainders=True):
    # Parse the order of the operations into a z3 AST using the Shunting-Yard Algorithm
    pres = {'+': 2, '-': 2, '*': 3, '/': 3}
    opmap = {'+': operator.add, '-': operator.sub, '*': operator.mul, '/': operator.truediv}

    consts = set()
    z3vars = dict()
    eqs = []
    for expr in exprs:
        opstack = []
        result = []
        for tok in expr[:-1]:
            if tok in pres:
                while opstack and pres[opstack[-1]] >= pres[tok]:
                    arg1,arg2 = result.pop(),result.pop()
                    op = opstack.pop()
                    result.append(opmap[op](arg2,arg1))
                    if no_remainders and op == '/':
                        eqs.append(arg2 % arg1 == 0)
                opstack.append(tok)
            else:
                if tok.startswith('x'):
                    if tok not in z3vars:
                        # BitVec is far faster than Int, I believe because Int is unbounded
                        z3vars[tok] = z3.BitVec(tok, 32)
                    result.append(z3vars[tok])
                else:
                    result.append(int(tok))
                    consts.add(int(tok))
        while opstack:
            arg1,arg2 = result.pop(),result.pop()
            op = opstack.pop()
            result.append(opmap[op](arg2,arg1))
            if no_remainders and op == '/':
                eqs.append(arg2 % arg1 == 0)
        eqs.append(operator.eq(result.pop(), int(expr[-1])))
    return z3vars, eqs, consts

In [ ]:
def grid_to_z3(grid):
    expr_grid = add_grid_vars(grid)
    exprs = extract_exprs(expr_grid)
    return parse_z3_exprs(exprs)

In [ ]:
def setup_solver(z3vars, eqs, consts, maxv):
    solver = z3.Solver()
    solver.add(eqs)
    solver.add(z3.Distinct(list(z3vars.values())))
    for v in z3vars.values():
        solver.add(1 <= v)
        solver.add(v <= maxv)
    for v in z3vars.values():
        for const in consts:
            solver.add(v != const)
    return solver

## Display Results 

In [ ]:
def draw_solution(img, nx, dx, ox, ny, dy, oy, solution, grid, size, color=(255,0,0)):
    solved = img.copy()
    draw_solved = ImageDraw.Draw(solved)
    font = ImageFont.truetype("/System/Library/Fonts/Supplemental/Arial.ttf", size=size)

    for iy in range(ny):
        for ix in range(nx):
            if grid[iy][ix].startswith('x'):
                x, y = int(ox + dx * ix), int(oy + dy * iy)
                draw_solved.text((x,y), str(solution[grid[iy][ix]]), font=font, fill=color, anchor='mm')
    return solved

## Example 1: u/ThatMadSniper's from Reddit

Original post [here](https://www.reddit.com/r/theydidthemath/comments/1emaoko/request_maths_square_puzzle/)

### Open Data

In [ ]:
import requests
import io

In [ ]:
url = "https://preview.redd.it/request-maths-square-puzzle-v0-8w0pninkl8hd1.png?auto=webp&s=e158a5de22eae940b1889b3c9dbd2922202fedc3"
puzzle_img = Image.open(io.BytesIO(requests.get(url).content))
puzzle_img.resize((puzzle_img.size[0]//4, puzzle_img.size[1]//4))

In [ ]:
# Crop to contents and remove extraneous strip for better alignment
grid_img = puzzle_img.crop((50,0,puzzle_img.size[0]*0.95,puzzle_img.size[1]*0.7))
grid_img = Image.fromarray(np.array(grid_img.convert('L'))[:, (np.arange(grid_img.size[0]) < 736) | (744 < np.arange(grid_img.size[0]))]).convert('RGB')
grid_img.resize((grid_img.size[0]//2, grid_img.size[1]//2))

### Transcribe Image and Apply Corrections

In [ ]:
grid, (nx, dx, ox), (ny, dy, oy) = transcribe_grid(grid_img, minwidth=1/20)
nx,ny

In [ ]:
check_grid(grid_img, nx, dx, ox, ny, dy, oy).resize((grid_img.size[0]//3, grid_img.size[1]//3))

In [ ]:
for row in grid:
    print(' '.join(f"{c: >4}" for c in row))

In [ ]:
# Corrections
grid[1][2] = '/'
grid[2][5] = '/'
grid = [[c.replace('x','*') for c in row] for row in grid]

In [ ]:
for row in grid:
    print(' '.join(f"{c: >4}" for c in row))

### Setup Solver

In [ ]:
expr_grid = add_grid_vars(grid)

In [ ]:
exprs = extract_exprs(expr_grid)

In [ ]:
z3vars, eqs, consts = parse_z3_exprs(exprs)
eqs

In [ ]:
solver = setup_solver(z3vars, eqs, consts, (nx//2)**2)
solver

In [ ]:
%%time
solver.check()

### Return Solution

In [ ]:
model = solver.model()
solution = {k: model.evaluate(v).as_long() for k,v in z3vars.items()}
solution

In [ ]:
sol_img = draw_solution(grid_img, nx, dx, ox, ny, dy, oy, solution, expr_grid, 40)
sol_img.resize((sol_img.size[0]//2, sol_img.size[1]//2))

In [ ]:
for row in expr_grid[:-1]:
    print(' '.join(f"{solution[c] if c in solution else c: >2}" for c in row[:-1]))

In [ ]:
for row in expr_grid[:-1:2]:
    print(' '.join(f"{solution[c] if c in solution else c: >2}" for c in row[:-1:2]))

## Example 2: u/SKYY99999's from Reddit

Original post [here](https://www.reddit.com/r/puzzles/comments/1j42efs/need_help_solving_a_huge_math_square/).

### Open Data

In [ ]:
url = "https://c.l3n.co/i/YG2MNe.png"
puzzle_img = Image.open(io.BytesIO(requests.get(url).content))
puzzle_img.resize((puzzle_img.size[0]//2, puzzle_img.size[1]//2))

In [ ]:
grid_img = puzzle_img.crop((0,0,puzzle_img.size[0],puzzle_img.size[1]*0.75))
grid_img = Image.fromarray(np.array(grid_img.convert('L'))[:, (np.arange(grid_img.size[0]) < 679) | (685 < np.arange(grid_img.size[0]))]).convert('RGB')
grid_img.resize((grid_img.size[0]//2, grid_img.size[1]//2))

### Transcribe Image and Apply Corrections

In [ ]:
grid, (nx, dx, ox), (ny, dy, oy) = transcribe_grid(grid_img, minwidth=1/20)
nx,ny

In [ ]:
check_grid(grid_img, nx, dx, ox, ny, dy, oy).resize((grid_img.size[0]//2, grid_img.size[1]//2))

In [ ]:
for row in grid:
    print(' '.join(f"{c: >4}" for c in row))

In [ ]:
grid[0][9] = '/'
grid[3][0] = '/'
grid[3][6] = '/'
grid[4][5] = '/'
grid = [[c.replace('x','*') for c in row] for row in grid]

In [ ]:
for row in grid:
    print(' '.join(f"{c: >4}" for c in row))

### Setup Solver

In [ ]:
expr_grid = add_grid_vars(grid)

In [ ]:
exprs = extract_exprs(expr_grid)

In [ ]:
z3vars, eqs, consts = parse_z3_exprs(exprs)
eqs

In [ ]:
solver = setup_solver(z3vars, eqs, consts, (nx//2)**2)
solver

### Return Solution

In [ ]:
%%time
solver.check()

In [ ]:
model = solver.model()
solution = {k: model.evaluate(v).as_long() for k,v in z3vars.items()}
solution

In [ ]:
sol_img = draw_solution(grid_img, nx, dx, ox, ny, dy, oy, solution, expr_grid, 30)
sol_img

In [ ]:
for row in expr_grid[:-1]:
    print(' '.join(f"{solution[c] if c in solution else c: >2}" for c in row[:-1]))

In [ ]:
for row in expr_grid[:-1:2]:
    print(' '.join(f"{solution[c] if c in solution else c: >2}" for c in row[:-1:2]))

In [ ]:
solutions = []
while solver.check() == z3.sat:
    model = solver.model()
    solution = {k: model.evaluate(v).as_long() for k,v in z3vars.items()}
    solutions.append(solution)
    solver.add(z3.Or(*[v != model.evaluate(v) for v in z3vars.values()]))
    print(f'found {len(solutions)} solutions, latest:', {k: model.evaluate(v).as_long() for k,v in z3vars.items()})

In [ ]:
len(solutions)

In [ ]:
solutions